<a href="https://colab.research.google.com/github/subchina/LLM-Project/blob/main/Evals_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Prelminary

In [2]:
# Download model to evaluate from Google Drive using gdown

!gdown https://drive.google.com/uc?id=132W0Teu8Q8KFVkQ31FeMwlmjBiAf5G5C


Downloading...
From (original): https://drive.google.com/uc?id=132W0Teu8Q8KFVkQ31FeMwlmjBiAf5G5C
From (redirected): https://drive.google.com/uc?id=132W0Teu8Q8KFVkQ31FeMwlmjBiAf5G5C&confirm=t&uuid=d6440dfe-8bb0-458b-827c-00fee905a403
To: /content/full-finetuned-phi5.zip
100% 14.1G/14.1G [04:17<00:00, 54.7MB/s]


In [3]:
# Clone the LLM-Project repository from GitHub

!git clone https://github.com/subchina/LLM-Project.git

Cloning into 'LLM-Project'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 161 (delta 56), reused 16 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (161/161), 18.56 MiB | 6.98 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [ ]:
# Create an output directory if it doesn't exist

import os

if not os.path.isdir('content/output'):
  os.mkdir('content/output')


FileNotFoundError: [Errno 2] No such file or directory: 'content/output'

In [4]:
# Install required packages
!pip install adapters
!pip install peft
!pip install langchain_community
!pip install langchain
!pip install python-dotenv
!pip install safetensors


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 67.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.7/408.7 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstal

##Phi ("phein") Fully finetuned

In [5]:
# Unzip model folder

!unzip /content/full-finetuned-phi5.zip -d /content/full-finetued-phi5

Archive:  /content/full-finetuned-phi5.zip
  inflating: /content/full-finetued-phi5/configuration_phi3.py  
  inflating: /content/full-finetued-phi5/tokenizer.model  
  inflating: /content/full-finetued-phi5/model-00001-of-00004.safetensors  
  inflating: /content/full-finetued-phi5/added_tokens.json  
  inflating: /content/full-finetued-phi5/generation_config.json  
  inflating: /content/full-finetued-phi5/special_tokens_map.json  
  inflating: /content/full-finetued-phi5/modeling_phi3.py  
  inflating: /content/full-finetued-phi5/model-00003-of-00004.safetensors  
  inflating: /content/full-finetued-phi5/model.safetensors.index.json  
  inflating: /content/full-finetued-phi5/config.json  
  inflating: /content/full-finetued-phi5/model-00002-of-00004.safetensors  
  inflating: /content/full-finetued-phi5/model-00004-of-00004.safetensors  
  inflating: /content/full-finetued-phi5/tokenizer.json  
  inflating: /content/full-finetued-phi5/tokenizer_config.json  


In [ ]:
# Load the pre-trained model and tokenizer

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_path = "/content/full-finetued-phi5"

model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
# Define a sample input for inference
input_text = "What is AI?"
inputs = tokenizer(input_text, return_tensors="pt")

# Run inference
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=200)  # Generates a response with a max length of 50 tokens

# Decode and print the output
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Input:", input_text)
print("Output:", output_text)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
# Evaluate the model using the eval.py script

!LOCAL_HF_MODEL_DIR=/content/full-finetuned-phi python LLM-Project/eval.py -o ./output -d LLM-Project/questions.json -B local_hf -m "full-finetued-phi4" -b 36


/content/LLM-Project/eval.py:6: LangChainDeprecationWarning: Importing HuggingFacePipeline from langchain.llms is deprecated. Please replace deprecated imports:

>> from langchain.llms import HuggingFacePipeline

with new imports of:

>> from langchain_community.llms import HuggingFacePipeline
You can use the langchain cli to **automatically** upgrade many imports. Please see documentation here <https://python.langchain.com/docs/versions/v0_2/>
  from langchain.llms.huggingface_pipeline import HuggingFacePipeline
/usr/local/lib/python3.10/dist-packages/langchain/chat_models/__init__.py:33: LangChainDeprecationWarning: Importing chat models from langchain is deprecated. Importing from langchain will no longer be supported as of langchain==0.2.0. Please import from langchain-community instead:

`from langchain_community.chat_models import AzureChatOpenAI`.

To install langchain-community run `pip install -U langchain-community`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/la

In [ ]:
import torch
torch.cuda.empty_cache()


: 

In [ ]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


: 

###Extra

In [ ]:
!unzip /content/Phi-3-finetune.zip -d /content/phi-3-finetune

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Specify separate paths for the tokenizer and model
tokenizer_path = "Phi-3-mini-4k-instruct"
model_path = "Phi-3-mini-4k-instruct/checkpoint-1000"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, use_safetensors=True)


In [ ]:
save_path = "full-finetuned-phi"

tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

In [ ]:
import os
import zipfile
import sys

def zip_folder(folder_path: str, output_zip: str):
    total_files = sum([len(files) for _, _, files in os.walk(folder_path)])
    zipped_files = 0

    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)
                zipped_files += 1
                progress = (zipped_files / total_files) * 100
                sys.stdout.write(f'\rZipping progress: {progress:.2f}%')
                sys.stdout.flush()
    print('\nZipping completed.')

# Example usage
folder_to_zip = './finetuned_phi'
output_zip_file = 'finetuned_phi.zip'
zip_folder(folder_to_zip, output_zip_file)
